In [1]:
import os
import numpy as np
import pandas as pd
os.chdir("/content/drive/MyDrive/cs289/proj/code")


In [ ]:
pip install pysolnp

In [ ]:
from data_process import data_prep_task
from learners import *
from sl_task import sl_task
from screeners import Screen_univar, Screen_l1
from pipeline import Pipeline
from sl_vim import importance

In [4]:
"""
 import clean wash data
"""
X_t, X_v, Y_t, Y_v = data_prep_task('wash', test_size=0.3)

Categorical features imputed successfully.
Discrete numerical features imputed successfully.
Int values turned into float type successfully.
Continuous numerical features imputed successfully.
One hot code finished successfully.
Dataset is standardized.
Data pre-processing is finished.


In [5]:
"""
 fit sl
"""
# Step 1. define task
task = sl_task(X = X_t, Y = Y_t, family = "Gaussian", K = 10)

# Step 2. define candidate learners
L_glm = Lrnr_glm(sl_task = task)
L_lasso = Lrnr_glmnet(sl_task = task)
L_rf = Lrnr_rf(sl_task = task)
L_xgb = Lrnr_xgboost(sl_task = task, num_round = 200)

S_l1 = Screen_l1(sl_task = task, k = 20) # screener univariate
L_xgb100 = Lrnr_xgboost(sl_task = task, num_round = 100) # pipepline
learners = [S_l1, L_xgb100]
L_pipe = Pipeline(sl_task = task, learners = learners)

stack = [L_glm, L_lasso, L_rf, L_xgb, L_pipe]

# Step 3. initialize sl
sl = initialize_sl(sl_task = task, stack = stack, meta = "nnls")


In [6]:
sl.train()

Training on Fold 1 of 10 

Training on Fold 2 of 10 

Training on Fold 3 of 10 

Training on Fold 4 of 10 

Training on Fold 5 of 10 

Training on Fold 6 of 10 

Training on Fold 7 of 10 

Training on Fold 8 of 10 

Training on Fold 9 of 10 

Training on Fold 10 of 10 



In [7]:
# summary sl
sl.summary()

 Num. of cv folds:  10 
 Meta learner:  nnls 
 n:  3286 
 p:  58
  learner    weight   cv_risk
0     glm  0.050501  1.028768
1  glmnet  0.157417  1.023921
2      rf  0.406125  1.018602
3     xgb  0.300024  1.030285
4    pipe  0.099450  1.023088


In [8]:
# calculate validation risk with sl
y_hat = sl.predict(X_v)
square_error_loss(y_hat, Y_v)

1.0115757864991888

In [ ]:
# variable importance with sl
idx_rank, diff_rank = importance(X = X_t, 
                                 Y = Y_t, 
                                 stack = stack, 
                                 meta = "nnls", 
                                 family = "Gaussian", 
                                 K1 = 5, 
                                 K2 = 10)

In [14]:
# rank of feature index based on their importance
idx_rank

array([ 2, 45, 44, 38,  9,  1, 41,  0, 11, 54, 40, 43,  4, 14, 37, 13, 57,
        8, 27, 15, 47, 10, 12, 34, 36, 48,  5,  7, 50, 42, 29, 16, 20, 35,
       39, 56, 51,  3, 53,  6, 46, 21, 26, 31, 19, 55, 24, 25, 49, 17, 28,
       52, 18, 22, 32, 23, 30, 33])